In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.dates as mdates
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

unidf=pd.read_csv(r"C:\Users\hajar\Time-Series-Crime-Forecasting-Minneapolis-3\data\Univ_Date.csv")

## ARIMA
1. Split the data into a train and test using time-based splitting
2. Check Assumptions and make corrections if needed
3. Select ARIMA Model
4. Select Best ARIMA Model
4. Interpret

### Split Data

Partition the data set into 4 parts corresponding to 2019-2024. then create nested training sets.

In [ ]:
#Validation Set

#sort Data
#unidf=unidf.set_index('Date')
#unidf=unidf.sort_index()
#unidf

,Crime_Count
Date,
2019-01-01,88
2019-01-02,102
2019-01-03,133
2019-01-04,95
2019-01-05,83
...,...
2025-03-06,89
2025-03-07,89
2025-03-08,70


In [134]:
#Divide Data into 4 pairs of train and test sets
df19= unidf[unidf['Date'].dt.year == 2019].copy()
df20= unidf[unidf['Date'].dt.year == 2019].copy()
df21= unidf[unidf['Date'].dt.year == 2019].copy()
df22= unidf[unidf['Date'].dt.year == 2019].copy()
df23= unidf[unidf['Date'].dt.year == 2019].copy()
df24= unidf[unidf['Date'].dt.year == 2019].copy()

AttributeError: Can only use .dt accessor with datetimelike values

## STEP 1: Set 1 2019-2021
### Check Stationarity


In [ ]:
#Look at plot
fig, ax=plt.subplots(figsize=(20, 5))
plt.plot(train_set.Crime_Count); 
plt.xticks(rotation=10, fontsize=15)

years = mdates.YearLocator()  
ax.xaxis.set_major_locator(years)

ax.set_xlim('2019-01-01', '2024-12-31')
ax.set_title("\nNumber of Reported Crimes per Day in Minneapolis from 2019-2024\n",weight='bold',fontdict={'fontsize':20})
ax.set_xlabel("\nDate\n",fontdict={'fontsize':15})
ax.set_ylabel("\nNumber of Reported Crimes\n",fontdict={'fontsize':15})



In [ ]:
#ADF Test

### Differencing

In [ ]:
# ORIGINAL SERIES



### Determine p and q

In [ ]:
#ACF
#plot_acf(train_set.Crime_Count)
#PACF
#plot_pacf(train_set.Crime_Count,lags=30)

### Calculate test MSE

In [ ]:
#test MSE1

## STEP 2: Set 2 2019-2022
### Check Stationarity

In [ ]:
#Look at plot
fig, ax=plt.subplots(figsize=(20, 5))
plt.plot(train_set.Crime_Count); 
plt.xticks(rotation=10, fontsize=15)

years = mdates.YearLocator()  
ax.xaxis.set_major_locator(years)

ax.set_xlim('2019-01-01', '2024-12-31')
ax.set_title("\nNumber of Reported Crimes per Day in Minneapolis from 2019-2024\n",weight='bold',fontdict={'fontsize':20})
ax.set_xlabel("\nDate\n",fontdict={'fontsize':15})
ax.set_ylabel("\nNumber of Reported Crimes\n",fontdict={'fontsize':15})



In [ ]:
#ADF Test

### Differencing

### Determine p and q

In [ ]:
#ACF
#plot_acf(train_set.Crime_Count)
#PACF
#plot_pacf(train_set.Crime_Count,lags=30)

### Calculate test MSE

## STEP 2: Set 2 2019-2023
### Check Stationarity

In [ ]:
#Look at plot
fig, ax=plt.subplots(figsize=(20, 5))
plt.plot(train_set.Crime_Count); 
plt.xticks(rotation=10, fontsize=15)

years = mdates.YearLocator()  
ax.xaxis.set_major_locator(years)

ax.set_xlim('2019-01-01', '2024-12-31')
ax.set_title("\nNumber of Reported Crimes per Day in Minneapolis from 2019-2024\n",weight='bold',fontdict={'fontsize':20})
ax.set_xlabel("\nDate\n",fontdict={'fontsize':15})
ax.set_ylabel("\nNumber of Reported Crimes\n",fontdict={'fontsize':15})



### Fit ARIMA Model and compare to AutoARIMA

In [ ]:
#ARIMA Model

In [ ]:
#Auto ARIMA Model

### Calculate test MSE

In [ ]:
#calculate test MSE

### K-Folds? Approach using all-subset selection

### Fit ARIMA Model and compare to AutoARIMA

In [ ]:
##AutoARIMA

#split into 6-folds
tscv = TimeSeriesSplit(n_splits=6)

# Initialize lists to store evaluation metrics
mse_scores = []

# Iterate over train-test splits and train models
for train_index, test_index in tscv.split(unidf):
    train_data, test_data = unidf.iloc[train_index], unidf.iloc[test_index]

    # Fit ARIMA model
    model = ARIMA(train_data, )  # Example order for ARIMA
    fitted_model = model.fit()

    # Make predictions
    predictions = fitted_model.forecast(steps=len(test_data))

    # Calculate Mean Squared Error
    mse = mean_squared_error(test_data, predictions)
    mse_scores.append(mse)

    print(f'Mean Squared Error for current split: {mse}')

# Calculate average Mean Squared Error across all splits
average_mse = np.mean(mse_scores)
print(f'Average Mean Squared Error across all splits: {average_mse}')

### Fit Best Model and Forecast

In [ ]:
#Fit Best Model

In [ ]:
#Forecast 2025

## Random Forests Time Series Regression
1. Lagged predictors
2. Check Assumptions and make corrections if needed
3. Select ARIMA Model
4. Select Best ARIMA Model
4. Interpret